In [1]:
using Pkg
using JulES
using TuLiPa
using Dates, Statistics

In [10]:
TuLiPa.INCLUDEELEMENT[TuLiPa.TypeKey(JulES.ABSTRACT_INFLOW_MODEL, "TwoStateBucketIfm")] = JulES.includeTwoStateBucketIfm!

inflowpath = joinpath(dirname(dirname(pwd())), "input", "static_input") # path to ifm_elements data

ifm_elements = JulES.TuLiPa.getelements(JulES.JSON.parsefile(joinpath(inflowpath, "ifm_elements.json")), inflowpath)

TuLiPa.addscenariotimeperiod!(ifm_elements, "ScenarioTimePeriod", 
    TuLiPa.getisoyearstart(1991), 
    TuLiPa.getisoyearstart(2000))

modelobjects = getmodelobjects(ifm_elements)
ifm_model = modelobjects[TuLiPa.Id("AbstractInflowModel", "127.13")]

S0 = 0
G0 = 0
itp_Lday = ifm_model.handler.hist_Lday
itp_P = ifm_model.handler.hist_P
itp_T = ifm_model.handler.hist_T

start_with_buffer = ifm_model.handler.scen_start - Day(365)
days = Dates.value( Day(ifm_model.handler.scen_stop - start_with_buffer))
real_days = Dates.value( Day(ifm_model.handler.scen_stop - ifm_model.handler.scen_start))    
timepoints_start = days - real_days 
timepoints = collect((1: days))

P = Vector{Float64}([i for i in timepoints])
T = Vector{Float64}([i for i in timepoints])
Lday =Vector{Float64}([i for i in timepoints])
for i in timepoints
    start = ifm_model.handler.scen_start + Day(i - 1)
    P[i] = TuLiPa.getweightedaverage(itp_P, start, JulES.ONEDAY_MS_TIMEDELTA)
    T[i] = TuLiPa.getweightedaverage(itp_T, start, JulES.ONEDAY_MS_TIMEDELTA)
    Lday[i] = TuLiPa.getweightedaverage(itp_Lday, start, JulES.ONEDAY_MS_TIMEDELTA)
end

itp_method = JulES.SteffenMonotonicInterpolation()
itp_P = JulES.interpolate(timepoints, P, itp_method)
itp_T = JulES.interpolate(timepoints, T, itp_method)
itp_Lday = JulES.interpolate(timepoints, Lday, itp_method)

res = JulES.predict(ifm_model.handler.predictor, S0, G0, itp_Lday, itp_P, itp_T, timepoints);

(Q, _) = res
Q = Float64.(Q)
Q = Q[timepoints_start:end]
Q .= Q .* ifm_model.handler.m3s_per_mm

1 / mean(Q)

0.06619134772002751